In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install kmodes

In [7]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [8]:
!pip install lightgbm

In [9]:
!pip install imblearn

In [10]:
pip install sktime

In [11]:
pip install tsfresh

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series
from sktime.classification.interval_based import TimeSeriesForestClassifier


from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from kmodes.kprototypes import KPrototypes
from sklearn.cluster import KMeans

In [13]:
path = '/content/drive/MyDrive/OULAD_analysis/'
# path = '/content/drive/MyDrive/OULAD_analysis/'

assessments = pd.read_csv(path + "assessments.csv")

courses = pd.read_csv(path + "courses.csv")

student_assessment = pd.read_csv(path + "studentAssessment.csv")

student_info = pd.read_csv(path + "studentInfo.csv")

student_registration = pd.read_csv(path + "studentRegistration.csv")

student_vle = pd.read_csv(path + "studentVle.csv")

vle = pd.read_csv(path + "vle.csv")

domains=pd.DataFrame(
    {
        "code_module": ["AAA", "BBB", "CCC", "DDD", "EEE", "FFF", "GGG"],
        "domain": [
            "Social Sciences",
            "Social Sciences",
            "STEM",
            "STEM",
            "STEM",
            "STEM",
            "Social Sciences",
        ],
    },
)

In [14]:
# Handling weights for module GGG
assessments.loc[(assessments['code_module']=='GGG') & (assessments['weight']==0), 'weight'] = 11.11
# Handling weights for module FFF
assessments.loc[(assessments['code_module']=='FFF') & (assessments['weight']==0), 'weight'] = 14.29

# Early At-risk Prediction

In [15]:
def OrdMapping(df):

    imd_band_mapping = {'0-10%': 0,'10-20': 1,'20-30%': 2 , '30-40%':3,'40-50%': 4, '50-60%':5, '60-70%':6, '70-80%':7, '80-90%':8, '90-100%':9 }
    highest_education_mapping= {'No Formal quals':0, 'Lower Than A Level':2, 'A Level or Equivalent':3, 'HE Qualification':4,'Post Graduate Qualification':5}

    df['imd_band'] = df.imd_band.map(imd_band_mapping)
    df['highest_education'] = df.highest_education.map(highest_education_mapping)

    return df

In [16]:
# demographic cluster

cluster_stdInfo = student_info.copy()

# Assume df is already loaded and contains the required columns
categorical_columns = ['gender', 'highest_education', 'imd_band', 'age_band', 'disability', 'region']
numerical_columns = ['num_of_prev_attempts', 'studied_credits']

# Convert categorical columns to category dtypes
for col in categorical_columns:
    cluster_stdInfo[col] = cluster_stdInfo[col].astype('category')

# Creating a copy of the data for clustering that excludes the key columns
cluster_stdInfo = cluster_stdInfo[categorical_columns + numerical_columns].copy()
cluster_stdInfo[categorical_columns] = cluster_stdInfo[categorical_columns].apply(lambda x: x.cat.codes)

# Convert the DataFrame to a NumPy array
data_matrix = cluster_stdInfo.to_numpy()

kproto = KPrototypes(n_clusters=5, init='Cao', n_init=5, verbose=1)

clusters = kproto.fit_predict(data_matrix, categorical=[cluster_stdInfo.columns.get_loc(c) for c in categorical_columns])

student_info['demographic_cluster'] = clusters

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 16242, ncost: 4449164.511113244
Run: 1, iteration: 2/100, moves: 52, ncost: 4448765.918636573
Run: 1, iteration: 3/100, moves: 0, ncost: 4448765.918636573
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 6854, ncost: 12033361.136152184
Run: 2, iteration: 2/100, moves: 817, ncost: 12033355.87603484
Run: 2, iteration: 3/100, moves: 0, ncost: 12033355.87603484
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 3, iteration: 1/100, moves: 6654, ncost: 7220640.105116582
Run: 3, iteration: 2/100, moves: 36, ncost: 7219801.845967251
Run: 3, iteration: 3/100, moves: 1, ncost: 7219799.366009992
Run: 3, iteration: 4/100, moves: 0, ncost: 7219799.366009992
Init: initializing centroids
Init: initializing clusters
In

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.cluster import KMeans

# Define necessary functions
def OrdMapping(df):
    # Your ordinal encoding logic here
    return df

## early preds
cuts = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270]
results = []

for cut in cuts:

    print("Point in time:", cut)
    ## create final_weighted_score, perc_ontime_sub features
    student_assessment_cut = student_assessment[student_assessment.date_submitted <= cut]

    assessmentsCombinedDf = student_assessment_cut.merge(assessments, on=['id_assessment'])

    assessmentsCombinedDf['weighted_score'] = assessmentsCombinedDf['score'] * assessmentsCombinedDf['weight'] / 100
    assessmentsCombinedDf['sum_weighted_score'] = assessmentsCombinedDf.groupby(["code_module","code_presentation","id_student"])['weighted_score'].transform('sum')
    assessmentsCombinedDf['sum_weight'] = assessmentsCombinedDf.groupby(["code_module","code_presentation","id_student"])['weight'].transform('sum')

    assessmentsCombinedDf['OntimeSubmission'] = np.where(assessmentsCombinedDf["date_submitted"] <= assessmentsCombinedDf["date"], 1, 0)
    assessmentsCombinedDf['ontime_sum'] = assessmentsCombinedDf.groupby(["code_module","code_presentation","id_student"])['OntimeSubmission'].transform('sum')
    assessmentsCombinedDf['ontime_count'] = assessmentsCombinedDf.groupby(["code_module","code_presentation","id_student"])['OntimeSubmission'].transform('count')

    assessmentsCombinedDf = assessmentsCombinedDf.groupby(['code_module','code_presentation','id_student'], as_index=False).last()
    assessmentsCombinedDf['final_weighted_score'] = assessmentsCombinedDf['sum_weighted_score'] / assessmentsCombinedDf['sum_weight']
    assessmentsCombinedDf["perc_ontime_sub"] = assessmentsCombinedDf['ontime_sum'] / assessmentsCombinedDf['ontime_count']

    assessmentsCombinedDf.drop(["id_assessment", "score", "date", "weight", "weighted_score", "sum_weighted_score",
                                "sum_weight", "OntimeSubmission", "ontime_sum", "ontime_count", "date_submitted",
                                "assessment_type"], axis=1, inplace=True)

    # Create avgClicksPerDay feature
    student_vle_cut = student_vle[student_vle.date <= cut]

    vleCombinedDf = student_vle_cut.merge(vle, on=['code_module','code_presentation', 'id_site']).drop(['week_from', 'week_to'], axis=1)
    vleCombinedDf['avgClicksPerDay'] = (vleCombinedDf.groupby(['code_module','code_presentation','id_student'])['sum_click']
                                        .transform('mean'))
    vleCombinedDf = vleCombinedDf.groupby(['code_module','code_presentation','id_student'], as_index=False).first()
    vleCombinedDf.drop(["id_site", "date", "activity_type", "sum_click"], axis=1, inplace=True)

    # merging student_registration, courses and domains
    std_reg_courses = student_registration.merge(courses, on=["code_module","code_presentation"])
    std_reg_courses_domain = std_reg_courses.merge(domains, on="code_module")
    std_reg_courses_domain.drop(['date_registration','date_unregistration'], axis=1, inplace=True)

    # merging student_info, std_reg_courses_domain, vleCombinedDf, assessmentsCombinedDf
    std_info_reg_courses_domain = student_info.merge(std_reg_courses_domain, on=["code_module", "code_presentation", "id_student"])
    std_info_reg_courses_domain_vle = std_info_reg_courses_domain.merge(vleCombinedDf, on=["code_module", "code_presentation", "id_student"])
    finalCombinedDf = std_info_reg_courses_domain_vle.merge(assessmentsCombinedDf, on=["code_module", "code_presentation", "id_student"])

    # Missing value handling
    finalCombinedDf.loc[(finalCombinedDf.final_weighted_score.isna()), 'final_weighted_score'] = 0
    finalCombinedDf.loc[(finalCombinedDf.imd_band.isna()), 'imd_band'] = '50-60%'

    # behavioural cluster
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
    finalCombinedDf['behavioural_cluster'] = kmeans.fit_predict(finalCombinedDf[['avgClicksPerDay']])


    #assesments cluster
    finalCombinedDf['assesments_cluster'] = kmeans.fit_predict(finalCombinedDf[['perc_ontime_sub',
                                                                            'final_weighted_score']])

    # Encoding
    ordinal_features = ['imd_band','highest_education']
    nominal_features = ['code_module','code_presentation','gender','region','age_band','disability']

    # ordinal encoding
    ordMapped_df = OrdMapping(finalCombinedDf)

    # One hot encoding
    onehotEncoded_df = pd.get_dummies(ordMapped_df, columns=nominal_features, drop_first=True, dtype=float)

    # Target variable creation
    onehotEncoded_df['is_At_Risk'] = np.where(onehotEncoded_df["final_result"].isin(["Withdrawn", "Fail"]), 1, 0)

    # final df
    features = ['avgClicksPerDay', 'final_weighted_score', 'perc_ontime_sub', 'demographic_cluster',
                'behavioural_cluster', 'assesments_cluster']

    target = ['is_At_Risk']

    final_df = onehotEncoded_df[features + target]

    # Scoring
    scoring = {'accuracy' : make_scorer(accuracy_score),
              'precision' : make_scorer(precision_score),
              'recall' : make_scorer(recall_score),
              'f1_score' : make_scorer(f1_score),
               'roc_auc' : 'roc_auc'}

    # load dataset
    X_ = final_df[features].values
    y = final_df.is_At_Risk.values

    # prepare models
    # DT
    # RF
    # XGB
    # LGBM
    models = []
    models.append(('DT', DecisionTreeClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('XGBoost', XGBClassifier()))
    models.append(('LightGBM', LGBMClassifier(force_row_wise=True, verbose=-1, verbose_eval=False)))
    models.append(('TSF', TimeSeriesForestClassifier(n_estimators=100, random_state=42)))

    # evaluate each model in turn

    for name, model in models:
        kfold = KFold(n_splits=5)
        cv_results = cross_validate(model, X_, y, cv=kfold, scoring=scoring)

        mean_accuracy = np.mean(cv_results["test_accuracy"])
        mean_precision = np.mean(cv_results["test_precision"])
        mean_recall = np.mean(cv_results["test_recall"])
        mean_f1_score = np.mean(cv_results["test_f1_score"])
        mean_roc_auc = np.mean(cv_results["test_roc_auc"])

        results.append([cut, name, "No", mean_accuracy, mean_precision, mean_recall, mean_f1_score, mean_roc_auc])

    # Scoring with sampling
    for name, model in models:

        imba_pipeline = make_pipeline(SMOTE(random_state=42),
                                      model)

        kfold = KFold(n_splits=5)
        cv_sampled_results = cross_validate(imba_pipeline, X_, y, cv=kfold, scoring=scoring)

        mean_accuracy = np.mean(cv_sampled_results["test_accuracy"])
        mean_precision = np.mean(cv_sampled_results["test_precision"])
        mean_recall = np.mean(cv_sampled_results["test_recall"])
        mean_f1_score = np.mean(cv_sampled_results["test_f1_score"])
        mean_roc_auc = np.mean(cv_sampled_results["test_roc_auc"])

        results.append([cut, name, "Yes", mean_accuracy, mean_precision, mean_recall, mean_f1_score, mean_roc_auc])


Point in time: 0
Point in time: 30
Point in time: 60
Point in time: 90
Point in time: 120
Point in time: 150
Point in time: 180
Point in time: 210
Point in time: 240
Point in time: 270


In [18]:
pd.DataFrame(data = results, columns=["Cut", "Model", "Sampling","mean_accuracy", "mean_precision",
                                      "mean_recall", "mean_f1_score", "mean_roc_auc"])

,Cut,Model,Sampling,mean_accuracy,mean_precision,mean_recall,mean_f1_score,mean_roc_auc
0,0,DT,No,0.553864,0.597224,0.614288,0.602907,0.547377
1,0,RF,No,0.607601,0.638291,0.687178,0.657882,0.608716
2,0,XGBoost,No,0.576813,0.612074,0.630064,0.619323,0.599686
3,0,LightGBM,No,0.597912,0.628871,0.663359,0.644715,0.618768
4,0,TSF,No,0.540330,0.582380,0.611891,0.593464,0.564203
...,...,...,...,...,...,...,...,...
95,270,DT,Yes,0.632151,0.534423,0.687236,0.595898,0.633021
96,270,RF,Yes,0.692904,0.618029,0.693788,0.638991,0.769271
97,270,XGBoost,Yes,0.690232,0.617807,0.744013,0.658418,0.788852
98,270,LightGBM,Yes,0.691278,0.625316,0.743960,0.657285,0.797009


In [19]:
!pip install openpyxl

In [20]:
pd.DataFrame(data = results, columns=["Cut", "Model", "Sampling","mean_accuracy", "mean_precision",
                                      "mean_recall", "mean_f1_score", "mean_roc_auc"]).to_excel("results.xlsx")